# Exploring Debugging Symbols

CFiddle can extract [DWARF](https://dwarfstd.org/doc/DWARF4.pdf) debugging information for compiled code.

In [4]:
from cfiddle import *
configure_for_jupyter()

In [5]:
source = code(r"""
extern "C" 
int foo(int a) {
    int k= 0; 
    return k;
}""")
b = build(source)

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
print(b[0].debug_info())

[  11] DW_TAG_compile_unit
[  12]   DW_AT_producer = b'GNU C++11 9.3.0 -mtune=generic -march=x86-64 -g3 -std=gnu++11 -fPIC -fasynchronous-unwind-tables -fstack-protector-strong -fstack-clash-protection -fcf-protection'
[  16]   DW_AT_language = 4
[  17]   DW_AT_name = b'.cfiddle/builds/anonymous_code/8cf18fa515d3e93f3ba23e44c931a328.cpp'
[  21]   DW_AT_comp_dir = b'/cse142L/fiddle/examples'
[  25]   DW_AT_low_pc = 4345
[  33]   DW_AT_high_pc = 23
[  41]   DW_AT_stmt_list = 0
[  45]   DW_AT_GNU_macros = 0
[  49]   DW_TAG_subprogram
[  50]     DW_AT_external = True
[  50]     DW_AT_name = b'foo'
[  54]     DW_AT_decl_file = 1
[  55]     DW_AT_decl_line = 3
[  56]     DW_AT_decl_column = 5
[  57]     DW_AT_type = 110
[  61]     DW_AT_low_pc = 4345
[  69]     DW_AT_high_pc = 23
[  77]     DW_AT_frame_base = [156]
[  79]     DW_AT_GNU_all_call_sites = True
[  79]     DW_AT_sibling = 110
[  83]     DW_TAG_formal_parameter
[  84]       DW_AT_name = b'a'
[  86]       DW_AT_decl_file = 1
[  87]